<a href="https://colab.research.google.com/github/guscldns/TestProject/blob/main/%EA%B3%BC%EC%A0%9C/poly_encoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 5기분들 코드

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 52.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 104.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.8 MB/s eta 0:00:00


In [2]:
!git clone https://github.com/doh0106/Poly-Encoder.git

Cloning into 'Poly-Encoder'...
remote: Enumerating objects: 368, done.
remote: Counting objects: 100% (360/360), done.
remote: Compressing objects: 100% (217/217), done.
remote: Total 368 (delta 197), reused 291 (delta 138), pack-reused 8
Receiving objects: 100% (368/368), 83.34 MiB | 13.64 MiB/s, done.
Resolving deltas: 100% (199/199), done.


In [3]:
%cd /content/Poly-Encoder/models/bert
# kobert 모델
!gdown 1R3QB2KADpICk0csnS_nEH5tAEl4FkaCP

# # !mkdir /content/Poly-Encoder/models/roberta
# %cd /content/Poly-Encoder/models/roberta
# # robert 모델
# !gdown 1QUk_Q6gGSRebFVWfjuPrsy6RAn4OZwv-

/content/Poly-Encoder/models/bert
Downloading...
From: https://drive.google.com/uc?id=1R3QB2KADpICk0csnS_nEH5tAEl4FkaCP
To: /content/Poly-Encoder/models/bert/pytorch_model.bin
100% 589M/589M [00:07<00:00, 74.0MB/s]


In [4]:
%cd /content/Poly-Encoder

/content/Poly-Encoder


In [18]:
# train poly_encoder
!python utils/run.py \
--model_type bert \
--bert_model models/bert/ \
--output_dir result/train1 \
--train_dir datasets/ \
--train_file dasan_train_data.pickle \
--valid_file dasan_train_data.pickle \
--use_pretrain \
--architecture poly \
--poly_m 16 \
--train_batch_size 2 \
--eval_batch_size 2 \
--max_contexts_length 256 \
--max_response_length 64 \
--num_train_epochs 1

2023-08-30 12:57:27.176070: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Namespace(bert_model='models/bert/', eval=False, model_type='bert', output_dir='result/train1', train_dir='datasets/', train_file='dasan_train_data.pickle', valid_file='dasan_train_data.pickle', test_file='test.pickle', neg_size=15, use_pretrain=True, architecture='poly', max_contexts_length=256, max_response_length=64, train_batch_size=2, eval_batch_size=2, print_freq=100, poly_m=16, learning_rate=5e-05, weight_decay=0.01, warmup_steps=100, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=1.0, seed=12345, gradient_accumulation_steps=1, fp16=False, fp16_opt_level='O1', gpu=0)
Train dir: datasets/
Output dir: result/train1
100% 884/884 [00:01<00:00, 571.23it/s]
100% 884/884 [00:02<00:00, 355.11it/s]
Loading parameters from models/bert/pytorch_model.bin
Some weights of BertModel were not initialized from the model checkpoint at models/bert/ and are newly init

In [ ]:
# !python3 /content/Poly-Encoder/utils/train.ipynb -bert_model /your/pretrained/model/dir/content/Poly-Encoder/models/bert/pytorch_model.bin --output_dir /your/ckpt/dir --train_dir /your/data/dir --use_pretrain --architecture poly --poly_m 16

In [19]:
# train cross_encoder
!python utils/run.py \
--model_type bert \
--bert_model models/bert/ \
--output_dir result/train2 \
--train_dir datasets/ \
--train_file dasan_train_data.pickle \
--valid_file dasan_train_data.pickle \
--use_pretrain \
--architecture cross \
--train_batch_size 2 \
--eval_batch_size 2 \
--max_contexts_length 256 \
--max_response_length 64 \
--num_train_epochs 1

2023-08-30 13:02:24.746225: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Namespace(bert_model='models/bert/', eval=False, model_type='bert', output_dir='result/train2', train_dir='datasets/', train_file='dasan_train_data.pickle', valid_file='dasan_train_data.pickle', test_file='test.pickle', neg_size=15, use_pretrain=True, architecture='cross', max_contexts_length=256, max_response_length=64, train_batch_size=2, eval_batch_size=2, print_freq=100, poly_m=0, learning_rate=5e-05, weight_decay=0.01, warmup_steps=100, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=1.0, seed=12345, gradient_accumulation_steps=1, fp16=False, fp16_opt_level='O1', gpu=0)
Train dir: datasets/
Output dir: result/train2
100% 884/884 [00:01<00:00, 558.21it/s]
100% 884/884 [00:01<00:00, 578.37it/s]
Loading parameters from models/bert/pytorch_model.bin
Some weights of BertModel were not initialized from the model checkpoint at models/bert/ and are newly init

In [9]:
pwd

'/content/Poly-Encoder'

In [24]:
from utils.model_for_inference import Load_Model_Tokenizer
import os
import pickle

import torch

CATEGORY = 'corona' # normal, water, corona

poly_dir = '/content/Poly-Encoder/result/train1'
cross_dir = '/content/Poly-Encoder/result/train2'
emb_dir = '/content/Poly-Encoder/datasets'
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# q1, q2, q3, q4, text, embedding(text에 대한) 저장 pickle / 하지만 현재는 inference에 text와 embedding만 사용
# with open(os.path.join(emb_dir, f'{CATEGORY}_with_text.pickle'), 'rb') as f:
with open(os.path.join(emb_dir, f'{CATEGORY}_with_text.pickle'), 'rb') as f:
    embedding_df = pickle.load(f)

cross_encoder, _ = Load_Model_Tokenizer(cross_dir, model_type='cross')
poly_encoder, tokenizer = Load_Model_Tokenizer(poly_dir, model_type='poly')
cross_encoder.to(device)
poly_encoder.to(device)


/content/Poly-Encoder/result/train2/pytorch_model.bin


You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embeding dimension will be 42001. This might induce some performance reduction as *Tensor Cores* will not be available. For more details  about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


모델 준비 끝
/content/Poly-Encoder/result/train1/pytorch_model.bin


You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embeding dimension will be 42001. This might induce some performance reduction as *Tensor Cores* will not be available. For more details  about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


모델 준비 끝


PolyEncoder(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(42001, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
       

In [25]:
# # 코드 셀 <undefined>
# # # %% [code]
from utils.inference import Callcenter
import numpy as np

call_center = Callcenter(poly_encoder=poly_encoder, cross_encoder=cross_encoder,
                        tokenizer=tokenizer, emb_df=embedding_df, device=device, topk=5)

query = '집에 가고 싶다.'
top_k_cross_scores, top_k_indices= call_center.inference(query)
# poly score가 가장 높은 k개의 답변 값들의 cross score 값과 index들 반환

top_cross_idx = top_k_indices[np.argmax(top_k_cross_scores)]
# k개의 답변들 중 cross score가 가장 높은 idx이용하여 최종 답변 산출
answer = embedding_df['text'].iloc[top_cross_idx]

print(f'질문 : {query}')
print(f'답변 : {answer}')

질문 : 집에 가고 싶다.
답변 :   네, 코로나십구 장기화로 매출 타격을 받고 있는 소상공인과 생계에 어려움을 겪는 취약계층 가구를 대상으로 도시가스 요금 납부 부담을 완화해드립니다.


In [28]:
# # 코드 셀 <undefined>
# # # %% [code]
from utils.inference import Callcenter
import numpy as np

call_center = Callcenter(poly_encoder=poly_encoder, cross_encoder=cross_encoder,
                        tokenizer=tokenizer, emb_df=embedding_df, device=device, topk=5)

query = '코로나19.'
top_k_cross_scores, top_k_indices= call_center.inference(query)
# poly score가 가장 높은 k개의 답변 값들의 cross score 값과 index들 반환

top_cross_idx = top_k_indices[np.argmax(top_k_cross_scores)]
# k개의 답변들 중 cross score가 가장 높은 idx이용하여 최종 답변 산출
answer = embedding_df['text'].iloc[top_cross_idx]

print(f'질문 : {query}')
print(f'답변 : {answer}')

질문 : 코로나19.
답변 :   네, 코로나십구 장기화로 매출 타격을 받고 있는 소상공인과 생계에 어려움을 겪는 취약계층 가구를 대상으로 도시가스 요금 납부 부담을 완화해드립니다.


In [26]:
# infrence
!python utils/text_2_emb.py \
--model_type bert \
--bert_model models/bert \
--text_path /path/to/카테고리별답변들.txt \
--output_dir /path/to/카테고리별embedding.pickle \
--gpu 1

Namespace(bert_model='models/bert', text_path='/path/to/카테고리별답변들.txt', max_response_length=128, output_dir='/path/to/카테고리별embedding.pickle', model_type='bert', gpu=1)
Loading parameters from models/bert/pytorch_model.bin
You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embeding dimension will be 42001. This might induce some performance reduction as *Tensor Cores* will not be available. For more details  about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc
Traceback (most recent call last):
  File "/content/Poly-Encoder/utils/text_2_emb.py", line 84, in <module>
    model.load_state_dict(model_state_dict)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 2041, in load_state_dict
    raise RuntimeError('Error(s) in loading state_dict for {}:\n\t{}'.f

### 원본 코드

In [ ]:
parlai interactive -m transformer/polyencoder \
    -mf zoo:pretrained_transformers/model_poly/model \
    --encode-candidate-vecs true \
    --eval-candidates fixed  \
    --fixed-candidates-path data/models/pretrained_transformers/convai_trainset_cands.txt

### 다른 git코드

In [ ]:
!git clone

fatal: You must specify a repository to clone.

usage: git clone [<options>] [--] <repo> [<dir>]

    -v, --verbose         be more verbose
    -q, --quiet           be more quiet
    --progress            force progress reporting
    --reject-shallow      don't clone shallow repository
    -n, --no-checkout     don't create a checkout
    --bare                create a bare repository
    --mirror              create a mirror repository (implies bare)
    -l, --local           to clone from a local repository
    --no-hardlinks        don't use local hardlinks, always copy
    -s, --shared          setup as shared repository
    --recurse-submodules[=<pathspec>]
                          initialize submodules in the clone
    --recursive ...       alias of --recurse-submodules
    -j, --jobs <n>        number of submodules cloned in parallel
    --template <template-directory>
                          directory from which templates will be used
    --reference <repo>    reference rep

In [ ]:
python3 run.py --bert_model bert_model/ --output_dir output_dstc7/ --train_dir dstc7/ --use_pretrain --architecture poly --poly_m 16 --eval